## Most common Spark commands

#### в данном ноутбуке собраны основные фичи для новичков
#### я написал его в первые дни работы в Сбере, когда сам изучал Spark.
#### что-то дополнял уже впоследствии

* 1) create dataframe
* 2) joins
* 3) execution plan
* 4) repartition vs partitionBy
* 5) write table
* 6) Regex
* 7) date
* 8) windows func
* 9) UDF
* 10) Broadcast filtering
* 11) Работа с HDFS

In [1]:
import pandas as pd

from datetime import datetime as dt
import time

from collections import OrderedDict, namedtuple
import tqdm

In [2]:
# Настройки интерфейса
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:98% !important; margin-left:1% !important; margin-right:auto !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_rows', 70)

import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import sys

In [4]:
# [рабочая] Конфигурация для ЛД 3.0 (Кластер 2.2)
spark_home = '/opt/cloudera/parcels/SPARK2/lib/spark2'
os.environ['SPARK_HOME'] = spark_home
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
os.environ['PYSPARK_PYTHON'] = '/opt/cloudera/parcels/PYENV.ZNO20008661/bin/python'
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'
 
sys.path.insert(0, os.path.join (spark_home,'python'))
sys.path.insert(0, os.path.join (spark_home,'python/lib/py4j-0.10.7-src.zip'))

In [5]:
from pyspark import SparkContext, SparkConf, HiveContext

In [7]:
SparkAppName = 'test_cherniaev_aa'

conf = None
conf = SparkConf().setAppName(SparkAppName.format(str(dt.now()).split()[1][:5])).\
    setMaster("yarn-client").\
    set('spark.dynamicAllocation.enabled', 'false').\
    set('spark.executor.memory','6g').\
    set('spark.driver.memory','4g').\
    set('spark.driver.maxResultSize','5g').\
    set('spark.executor.instances', '10').\
    set('spark.executor.cores', '2').\
    set('spark.yarn.executor.memoryOverhead', '2g') .\
    set('spark.yarn.driver.memoryOverhead', '2g') .\
    set('spark.sql.orc.enabled','true').\
    set('spark.sql.orc.filterPushdown','true').\
    set('spark.sql.parquet.filterPushdown','true').\
    set('spark.sql.autoBroadcastJoinThreshold','0').\
    set('spark.sql.hive.convertMetastoreOrc','true').\
    set('spark.ui.port', '4713').\
    set('spark.port.maxRetries', '113').\
    set('spark.sql.hive.caseSensitiveInferenceMode', 'INFER_ONLY').\
    set('spark.kryoserializer.buffer.max','1024m').\
    set('spark.yarn.am.memory','10g').\
    set('spark.local.dir', 'sparktmp').\
    set('spark.core.connection.ack.wait.timeout', '800s').\
    set('spark.storage.blockManagerSlaveTimeoutMs', '800s').\
    set('spark.shuffle.io.connectionTimeout', '800s').\
    set('spark.rpc.askTimeout', '800s').\
    set('spark.network.timeout', '3600s').\
    set('spark.rpc.lookupTimeout', '800s').\
    set('spark.shuffle.consolidateFiles', 'true').\
    set('spark.yarn.jars','{}/jars/*.jar'.format(spark_home)).\
    set('spark.shuffle.filebuffer', '64k')
#         set('spark.sql.shuffle.partitions','100').\    

#     set('spark.yarn.queue','root.g_dl_u_corp.korsakov_dot_n_dot_n').\
#     set('spark.yarn.queue','root.z_transaction').\
#     set('spark.yarn.queue','root.g_dl_u_corp.korsakov-nn_ca-sbrf-ru').\

#  set('spark.hadoop.hive.metastore.uris', 'thrift://februs7.lab.df.sbrf.ru:9083').\ - репозиторий ЛД 20 (ссылки на схемы - без dl_)
#  set('hive.metastore.uris', 'thrift://pkli-chd1073.labiac.df.sbrf.ru:48869').\ - репозиторий динамичИнфры ЛД 30 (нужны приставки dl_)

#     set('spark.sql.autoBroadcastJoinThreshold','314572800').\
#     set('spark.hadoop.hive.metastore.uris', 'thrift://februs7.lab.df.sbrf.ru:9083').\

In [8]:
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [9]:
# sparkContext для работы с датафреймами
# HiveContext для последующего применения чистого sql 
st = dt.now(); print('Start\t\t', st)
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = HiveContext(sc); et = dt.now(); dd = et - st; stt = et
print('Allocated\t', et, '| dt=', dd )

Start		 2021-04-23 10:41:56.297022
Allocated	 2021-04-23 10:42:49.521735 | dt= 0:00:53.224713


In [26]:
#sc.stop()

#### Константы

In [10]:
PREFIX_TMP = 'tmp_'
PREFIX_SBX = 'sbx_'
PREFIX_LD20 = 'dl_' # префикс для доступа к схеме на ЛД20 из кластера ЛД30
PREFIX_COPY_TBL = 'xx_' # Префикс пренесенных таблиц

#### Создание тестовых DF

In [324]:
data4del_lst = [
    Row(description='hjg', id='01', city='Moscow',), 
    Row(description='uuu', id='02', city='Moscow',), 
    Row(description='kjh', id='03', city='Moscow',), 
    Row(description='aaa', id='04', city='St-Petersburg',),
    Row(description='lij', id='01', city='Moscow',), 
    Row(description='bbb', id='02', city='Moscow',),
    Row(description='fff', id='03', city='St-Petersburg',), 
    Row(description='ppp', id='04', city='St-Petersburg',),
    Row(description='jjj', id='05', city='St-Petersburg',),]
partition_df = sqlContext.createDataFrame(data4del_lst)
partition_df.show()

+-------------+-----------+---+
|         city|description| id|
+-------------+-----------+---+
|       Moscow|        hjg| 01|
|       Moscow|        uuu| 02|
|       Moscow|        kjh| 03|
|St-Petersburg|        aaa| 04|
|       Moscow|        lij| 01|
|       Moscow|        bbb| 02|
|St-Petersburg|        fff| 03|
|St-Petersburg|        ppp| 04|
|St-Petersburg|        jjj| 05|
+-------------+-----------+---+



In [325]:
# изменение названий колонок
new_names = ['city_name', 'descr', 'id']
DF_data = partition_df.toDF(*new_names)

In [326]:
DF_data.show()

+-------------+-----+---+
|    city_name|descr| id|
+-------------+-----+---+
|       Moscow|  hjg| 01|
|       Moscow|  uuu| 02|
|       Moscow|  kjh| 03|
|St-Petersburg|  aaa| 04|
|       Moscow|  lij| 01|
|       Moscow|  bbb| 02|
|St-Petersburg|  fff| 03|
|St-Petersburg|  ppp| 04|
|St-Petersburg|  jjj| 05|
+-------------+-----+---+



In [327]:
DF_data.select(*new_names).show()

+-------------+-----+---+
|    city_name|descr| id|
+-------------+-----+---+
|       Moscow|  hjg| 01|
|       Moscow|  uuu| 02|
|       Moscow|  kjh| 03|
|St-Petersburg|  aaa| 04|
|       Moscow|  lij| 01|
|       Moscow|  bbb| 02|
|St-Petersburg|  fff| 03|
|St-Petersburg|  ppp| 04|
|St-Petersburg|  jjj| 05|
+-------------+-----+---+



In [328]:
# создание через RDD и затем в toDF
col_names = [('10', '  text  '),
             ('20','  t e x t '), 
             ('31',' tx  t '), 
             ('10',' еще один '), 
             ('10', '  снова text  '),
             ('20','  опять t e x t '),
            ]
tt = sc.parallelize(col_names).toDF(['k','v'])
tt.show()

+---+----------------+
|  k|               v|
+---+----------------+
| 10|          text  |
| 20|        t e x t |
| 31|          tx  t |
| 10|       еще один |
| 10|    снова text  |
| 20|  опять t e x t |
+---+----------------+



In [329]:
# напрямую указание схемы 

ul_data_schema = StructType([
  StructField('ul_inn_ul', StringType(), True), 
  StructField('val_ul', LongType(), True),
])

 
ul_data = [('Q1', 1), ('W1',10), ('Q1', 3), ('Q1', 7), ('W1', 20)]

ul_tt = sqlContext.createDataFrame(ul_data, schema=ul_data_schema)

ul_tt.show()

+---------+------+
|ul_inn_ul|val_ul|
+---------+------+
|       Q1|     1|
|       W1|    10|
|       Q1|     3|
|       Q1|     7|
|       W1|    20|
+---------+------+



In [330]:
# метод через Row

data4del_lst = [
    Row(description= None, id=1, city='Paris',), 
    Row(description='область', id=2, city='Saratov',), 
    Row(description='город', id=3, city='Moscow',), 
    Row(description='город', id=4, city='St-Petersburg',),
    Row(description='город', id=5, city='Moscow',), 
    Row(description='область', id=6, city='Moscow',),
    Row(description='область', id=7, city='Cherepovets',), 
    Row(description='область', id=4, city='Volhov',),
    Row(description='область', id=5, city='Vologda',),]
test_df = sqlContext.createDataFrame(data4del_lst)
test_df.show()

+-------------+-----------+---+
|         city|description| id|
+-------------+-----------+---+
|        Paris|       null|  1|
|      Saratov|    область|  2|
|       Moscow|      город|  3|
|St-Petersburg|      город|  4|
|       Moscow|      город|  5|
|       Moscow|    область|  6|
|  Cherepovets|    область|  7|
|       Volhov|    область|  4|
|      Vologda|    область|  5|
+-------------+-----------+---+



### sql-like join VS df join

In [487]:
# на физическом плане джоины, написанные в sql и через датафреймы одинаковы в плане скорости
# Catalyst сведет все в физическом плане одинаково. SQL или DF выступают в роли API

test_df.join(ul_tt, test_df.id == ul_tt.val_ul).explain()

== Physical Plan ==
*SortMergeJoin [id#3525L], [val_ul#3515L], Inner
:- *Sort [id#3525L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(id#3525L, 200)
:     +- *Filter isnotnull(id#3525L)
:        +- Scan ExistingRDD[city#3523,description#3524,id#3525L]
+- *Sort [val_ul#3515L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(val_ul#3515L, 200)
      +- *Filter isnotnull(val_ul#3515L)
         +- Scan ExistingRDD[ul_inn_ul#3514,val_ul#3515L]


In [488]:
test_df.createOrReplaceTempView('tab1')
ul_tt.createOrReplaceTempView('tab2')

In [492]:
sqlContext.sql('select * from tab1 a join tab2 b on a.id == b.val_ul').explain()

== Physical Plan ==
*SortMergeJoin [id#3525L], [val_ul#3515L], Inner
:- *Sort [id#3525L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(id#3525L, 200)
:     +- *Filter isnotnull(id#3525L)
:        +- Scan ExistingRDD[city#3523,description#3524,id#3525L]
+- *Sort [val_ul#3515L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(val_ul#3515L, 200)
      +- *Filter isnotnull(val_ul#3515L)
         +- Scan ExistingRDD[ul_inn_ul#3514,val_ul#3515L]


### Pyspark Joins

#### Inner

In [331]:
df_inner = partition_df.join(test_df, on='id') 
df_inner.sort('id').show()


+---+-------------+-----------+-------------+-----------+
| id|         city|description|         city|description|
+---+-------------+-----------+-------------+-----------+
| 01|       Moscow|        hjg|        Paris|       null|
| 01|       Moscow|        lij|        Paris|       null|
| 02|       Moscow|        bbb|      Saratov|    область|
| 02|       Moscow|        uuu|      Saratov|    область|
| 03|       Moscow|        kjh|       Moscow|      город|
| 03|St-Petersburg|        fff|       Moscow|      город|
| 04|St-Petersburg|        ppp|       Volhov|    область|
| 04|St-Petersburg|        ppp|St-Petersburg|      город|
| 04|St-Petersburg|        aaa|St-Petersburg|      город|
| 04|St-Petersburg|        aaa|       Volhov|    область|
| 05|St-Petersburg|        jjj|       Moscow|      город|
| 05|St-Petersburg|        jjj|      Vologda|    область|
+---+-------------+-----------+-------------+-----------+



In [332]:
df_inner.explain()

== Physical Plan ==
*Project [id#3474, city#3472, description#3473, city#3523, description#3524]
+- *SortMergeJoin [cast(id#3474 as bigint)], [id#3525L], Inner
   :- *Sort [cast(id#3474 as bigint) ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(cast(id#3474 as bigint), 200)
   :     +- *Filter isnotnull(id#3474)
   :        +- Scan ExistingRDD[city#3472,description#3473,id#3474]
   +- *Sort [id#3525L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#3525L, 200)
         +- *Filter isnotnull(id#3525L)
            +- Scan ExistingRDD[city#3523,description#3524,id#3525L]


In [333]:
# бродкаст убирает шаффл, размещая одну из таблиц(маленький размер) на каждом из воркеров

df_inner_broadcast = partition_df.join(broadcast(test_df), on='id') 
df_inner_broadcast.sort('id').show()

+---+-------------+-----------+-------------+-----------+
| id|         city|description|         city|description|
+---+-------------+-----------+-------------+-----------+
| 01|       Moscow|        hjg|        Paris|       null|
| 01|       Moscow|        lij|        Paris|       null|
| 02|       Moscow|        uuu|      Saratov|    область|
| 02|       Moscow|        bbb|      Saratov|    область|
| 03|       Moscow|        kjh|       Moscow|      город|
| 03|St-Petersburg|        fff|       Moscow|      город|
| 04|St-Petersburg|        aaa|St-Petersburg|      город|
| 04|St-Petersburg|        ppp|St-Petersburg|      город|
| 04|St-Petersburg|        aaa|       Volhov|    область|
| 04|St-Petersburg|        ppp|       Volhov|    область|
| 05|St-Petersburg|        jjj|      Vologda|    область|
| 05|St-Petersburg|        jjj|       Moscow|      город|
+---+-------------+-----------+-------------+-----------+



In [334]:
df_inner_broadcast.select(F.count('id')).show()


+---------+
|count(id)|
+---------+
|       12|
+---------+



In [335]:
df_inner_broadcast.explain()

== Physical Plan ==
*Project [id#3474, city#3472, description#3473, city#3523, description#3524]
+- *BroadcastHashJoin [cast(id#3474 as bigint)], [id#3525L], Inner, BuildRight
   :- *Filter isnotnull(id#3474)
   :  +- Scan ExistingRDD[city#3472,description#3473,id#3474]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[2, bigint, false]))
      +- *Filter isnotnull(id#3525L)
         +- Scan ExistingRDD[city#3523,description#3524,id#3525L]


In [336]:
df_inner2 = partition_df.join(test_df).where(partition_df.id == test_df.id)
df_inner2.show()

+-------------+-----------+---+-------------+-----------+---+
|         city|description| id|         city|description| id|
+-------------+-----------+---+-------------+-----------+---+
|St-Petersburg|        jjj| 05|      Vologda|    область|  5|
|St-Petersburg|        jjj| 05|       Moscow|      город|  5|
|       Moscow|        lij| 01|        Paris|       null|  1|
|       Moscow|        hjg| 01|        Paris|       null|  1|
|St-Petersburg|        fff| 03|       Moscow|      город|  3|
|       Moscow|        kjh| 03|       Moscow|      город|  3|
|       Moscow|        uuu| 02|      Saratov|    область|  2|
|       Moscow|        bbb| 02|      Saratov|    область|  2|
|St-Petersburg|        ppp| 04|St-Petersburg|      город|  4|
|St-Petersburg|        ppp| 04|       Volhov|    область|  4|
|St-Petersburg|        aaa| 04|St-Petersburg|      город|  4|
|St-Petersburg|        aaa| 04|       Volhov|    область|  4|
+-------------+-----------+---+-------------+-----------+---+



In [337]:
df_inner.select(F.count('id')).show()

+---------+
|count(id)|
+---------+
|       12|
+---------+



In [338]:
df_inner.show()

+---+-------------+-----------+-------------+-----------+
| id|         city|description|         city|description|
+---+-------------+-----------+-------------+-----------+
| 05|St-Petersburg|        jjj|       Moscow|      город|
| 05|St-Petersburg|        jjj|      Vologda|    область|
| 01|       Moscow|        hjg|        Paris|       null|
| 01|       Moscow|        lij|        Paris|       null|
| 03|       Moscow|        kjh|       Moscow|      город|
| 03|St-Petersburg|        fff|       Moscow|      город|
| 02|       Moscow|        uuu|      Saratov|    область|
| 02|       Moscow|        bbb|      Saratov|    область|
| 04|St-Petersburg|        ppp|St-Petersburg|      город|
| 04|St-Petersburg|        ppp|       Volhov|    область|
| 04|St-Petersburg|        aaa|St-Petersburg|      город|
| 04|St-Petersburg|        aaa|       Volhov|    область|
+---+-------------+-----------+-------------+-----------+



In [339]:
df_inner = partition_df.join(test_df, on='id', how='inner')# тоже уберет дубль по id
df_inner.sort('id').show()

+---+-------------+-----------+-------------+-----------+
| id|         city|description|         city|description|
+---+-------------+-----------+-------------+-----------+
| 01|       Moscow|        hjg|        Paris|       null|
| 01|       Moscow|        lij|        Paris|       null|
| 02|       Moscow|        uuu|      Saratov|    область|
| 02|       Moscow|        bbb|      Saratov|    область|
| 03|       Moscow|        kjh|       Moscow|      город|
| 03|St-Petersburg|        fff|       Moscow|      город|
| 04|St-Petersburg|        aaa|St-Petersburg|      город|
| 04|St-Petersburg|        ppp|       Volhov|    область|
| 04|St-Petersburg|        aaa|       Volhov|    область|
| 04|St-Petersburg|        ppp|St-Petersburg|      город|
| 05|St-Petersburg|        jjj|       Moscow|      город|
| 05|St-Petersburg|        jjj|      Vologda|    область|
+---+-------------+-----------+-------------+-----------+



In [340]:
# Spark может вести себя некорректно, если join по полям с одинаковым названиям. (известный баг) 
# лучше менять названия столбцов по которым идет слияние
# протестировать, уйдет ли баг join-а по полю с одинаковым названием столбца.

df_inner = partition_df.join(test_df, partition_df.id==test_df.id.alias('id2'), how='inner') # здесь уже 2 столбца id из обоих таблиц
df_inner.show()

+-------------+-----------+---+-------------+-----------+---+
|         city|description| id|         city|description| id|
+-------------+-----------+---+-------------+-----------+---+
|St-Petersburg|        jjj| 05|      Vologda|    область|  5|
|St-Petersburg|        jjj| 05|       Moscow|      город|  5|
|       Moscow|        lij| 01|        Paris|       null|  1|
|       Moscow|        hjg| 01|        Paris|       null|  1|
|St-Petersburg|        fff| 03|       Moscow|      город|  3|
|       Moscow|        kjh| 03|       Moscow|      город|  3|
|       Moscow|        uuu| 02|      Saratov|    область|  2|
|       Moscow|        bbb| 02|      Saratov|    область|  2|
|St-Petersburg|        ppp| 04|       Volhov|    область|  4|
|St-Petersburg|        ppp| 04|St-Petersburg|      город|  4|
|St-Petersburg|        aaa| 04|       Volhov|    область|  4|
|St-Petersburg|        aaa| 04|St-Petersburg|      город|  4|
+-------------+-----------+---+-------------+-----------+---+



In [341]:
df_inner = partition_df.join(test_df, partition_df.id==test_df.id, how='inner')
df_inner.show()

+-------------+-----------+---+-------------+-----------+---+
|         city|description| id|         city|description| id|
+-------------+-----------+---+-------------+-----------+---+
|St-Petersburg|        jjj| 05|       Moscow|      город|  5|
|St-Petersburg|        jjj| 05|      Vologda|    область|  5|
|       Moscow|        lij| 01|        Paris|       null|  1|
|       Moscow|        hjg| 01|        Paris|       null|  1|
|       Moscow|        kjh| 03|       Moscow|      город|  3|
|St-Petersburg|        fff| 03|       Moscow|      город|  3|
|       Moscow|        uuu| 02|      Saratov|    область|  2|
|       Moscow|        bbb| 02|      Saratov|    область|  2|
|St-Petersburg|        aaa| 04|St-Petersburg|      город|  4|
|St-Petersburg|        aaa| 04|       Volhov|    область|  4|
|St-Petersburg|        ppp| 04|St-Petersburg|      город|  4|
|St-Petersburg|        ppp| 04|       Volhov|    область|  4|
+-------------+-----------+---+-------------+-----------+---+



#### Left / Right

In [342]:
df_left = partition_df.select('id', partition_df.city.alias('city_1'), partition_df.description.alias('desription_1')).join(test_df, on='id', how='left')
df_left.sort('id').show()

# используем alias, чтобы в новом df не было колонок с одинаковыми именами

+---+-------------+------------+-------------+-----------+
| id|       city_1|desription_1|         city|description|
+---+-------------+------------+-------------+-----------+
| 01|       Moscow|         lij|        Paris|       null|
| 01|       Moscow|         hjg|        Paris|       null|
| 02|       Moscow|         bbb|      Saratov|    область|
| 02|       Moscow|         uuu|      Saratov|    область|
| 03|St-Petersburg|         fff|       Moscow|      город|
| 03|       Moscow|         kjh|       Moscow|      город|
| 04|St-Petersburg|         ppp|       Volhov|    область|
| 04|St-Petersburg|         aaa|       Volhov|    область|
| 04|St-Petersburg|         ppp|St-Petersburg|      город|
| 04|St-Petersburg|         aaa|St-Petersburg|      город|
| 05|St-Petersburg|         jjj|       Moscow|      город|
| 05|St-Petersburg|         jjj|      Vologda|    область|
+---+-------------+------------+-------------+-----------+



In [343]:
df_right = partition_df.select('id', partition_df.city.alias('city_1'), partition_df.description.alias('desription_1')).join(test_df, on='id', how='right')
df_right.sort('id').show()

+---+-------------+------------+-------------+-----------+
| id|       city_1|desription_1|         city|description|
+---+-------------+------------+-------------+-----------+
|  1|       Moscow|         lij|        Paris|       null|
|  1|       Moscow|         hjg|        Paris|       null|
|  2|       Moscow|         uuu|      Saratov|    область|
|  2|       Moscow|         bbb|      Saratov|    область|
|  3|St-Petersburg|         fff|       Moscow|      город|
|  3|       Moscow|         kjh|       Moscow|      город|
|  4|St-Petersburg|         ppp|St-Petersburg|      город|
|  4|St-Petersburg|         ppp|       Volhov|    область|
|  4|St-Petersburg|         aaa|St-Petersburg|      город|
|  4|St-Petersburg|         aaa|       Volhov|    область|
|  5|St-Petersburg|         jjj|      Vologda|    область|
|  5|St-Petersburg|         jjj|       Moscow|      город|
|  6|         null|        null|       Moscow|    область|
|  7|         null|        null|  Cherepovets|    област

In [344]:
df_right.select(F.count('id')).show()

+---------+
|count(id)|
+---------+
|       14|
+---------+



In [345]:
df_right.select(F.countDistinct('id')).show()

+------------------+
|count(DISTINCT id)|
+------------------+
|                 7|
+------------------+



#### Full Outer

In [346]:
#избегаем портянок с alias
aliased_df = partition_df.select('id', \
                                 partition_df.city.alias('city_1'), \
                                 partition_df.description.alias('desription_1'))

In [347]:
df_full_outer = aliased_df.join(test_df, on='id', how='full')
df_full_outer.sort('id').show()

+---+-------------+------------+-------------+-----------+
| id|       city_1|desription_1|         city|description|
+---+-------------+------------+-------------+-----------+
| 01|       Moscow|         hjg|        Paris|       null|
| 01|       Moscow|         lij|        Paris|       null|
| 02|       Moscow|         uuu|      Saratov|    область|
| 02|       Moscow|         bbb|      Saratov|    область|
| 03|St-Petersburg|         fff|       Moscow|      город|
| 03|       Moscow|         kjh|       Moscow|      город|
| 04|St-Petersburg|         ppp|St-Petersburg|      город|
| 04|St-Petersburg|         aaa|St-Petersburg|      город|
| 04|St-Petersburg|         ppp|       Volhov|    область|
| 04|St-Petersburg|         aaa|       Volhov|    область|
| 05|St-Petersburg|         jjj|       Moscow|      город|
| 05|St-Petersburg|         jjj|      Vologda|    область|
|  6|         null|        null|       Moscow|    область|
|  7|         null|        null|  Cherepovets|    област

#### Semi / Anti

In [348]:
df_left_semi = aliased_df.join(test_df, on='id', how='left_semi')
df_left_semi.sort('id').show()

+---+-------------+------------+
| id|       city_1|desription_1|
+---+-------------+------------+
| 01|       Moscow|         hjg|
| 01|       Moscow|         lij|
| 02|       Moscow|         uuu|
| 02|       Moscow|         bbb|
| 03|St-Petersburg|         fff|
| 03|       Moscow|         kjh|
| 04|St-Petersburg|         aaa|
| 04|St-Petersburg|         ppp|
| 05|St-Petersburg|         jjj|
+---+-------------+------------+



In [349]:
df_left_anti = aliased_df.join(test_df, on='id', how='leftanti')
df_left_anti.show()

+---+------+------------+
| id|city_1|desription_1|
+---+------+------------+
+---+------+------------+



#### Cross

In [350]:
df_cross = aliased_df.crossJoin(test_df)
df_cross.show()

+---+------+------------+-------------+-----------+---+
| id|city_1|desription_1|         city|description| id|
+---+------+------------+-------------+-----------+---+
| 01|Moscow|         hjg|        Paris|       null|  1|
| 01|Moscow|         hjg|      Saratov|    область|  2|
| 01|Moscow|         hjg|       Moscow|      город|  3|
| 01|Moscow|         hjg|St-Petersburg|      город|  4|
| 01|Moscow|         hjg|       Moscow|      город|  5|
| 01|Moscow|         hjg|       Moscow|    область|  6|
| 01|Moscow|         hjg|  Cherepovets|    область|  7|
| 01|Moscow|         hjg|       Volhov|    область|  4|
| 01|Moscow|         hjg|      Vologda|    область|  5|
| 02|Moscow|         uuu|        Paris|       null|  1|
| 02|Moscow|         uuu|      Saratov|    область|  2|
| 02|Moscow|         uuu|       Moscow|      город|  3|
| 02|Moscow|         uuu|St-Petersburg|      город|  4|
| 02|Moscow|         uuu|       Moscow|      город|  5|
| 02|Moscow|         uuu|       Moscow|    облас

### План запроса explain() 

In [351]:
df_left_semi.explain()

== Physical Plan ==
SortMergeJoin [cast(id#3474 as bigint)], [id#3525L], LeftSemi
:- *Sort [cast(id#3474 as bigint) ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(cast(id#3474 as bigint), 200)
:     +- *Project [id#3474, city#3472 AS city_1#3743, description#3473 AS desription_1#3744]
:        +- *Filter isnotnull(id#3474)
:           +- Scan ExistingRDD[city#3472,description#3473,id#3474]
+- *Sort [id#3525L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(id#3525L, 200)
      +- *Project [id#3525L]
         +- Scan ExistingRDD[city#3523,description#3524,id#3525L]


In [352]:
df_cross.explain()

== Physical Plan ==
CartesianProduct
:- *Project [id#3474, city#3472 AS city_1#3743, description#3473 AS desription_1#3744]
:  +- Scan ExistingRDD[city#3472,description#3473,id#3474]
+- Scan ExistingRDD[city#3523,description#3524,id#3525L]


### repartition 
 - разделяет df на бакеты. в hdfs будет несколько файлов, в данном случае 3, сгруппированыые по id. те один и тот же id не будет в разных файлах.
 - также это сколько всего у нас будет партиций для физического исполнения на воркерах

### partitionBy
 - разобьет df на директории. физически будут выглядеть как папки в hdfs. сколько значений в столбце (у нас тут city) , столько и директорий. Подходит для низкой кардинальности
 -  динамическое (можно добавлять) / статическое нельзя

In [353]:
partition_df.repartition(3, 'id').write.format('csv').mode('overwrite').partitionBy('city').saveAsTable('{}.{}'.format('default', 'partition_table_csv'))

In [355]:
#show create table partition_df

In [356]:
tt2.write.format('csv').mode('overwrite').saveAsTable('{}.{}'.format('default', 'tablichka_csv'))

NameError: name 'tt2' is not defined

In [257]:
tt2.repartition(2,'rn_rec').write.format('csv').mode('overwrite').saveAsTable('{}.{}'.format('default', 'tablichka_csv_rep'))

In [256]:
tt3.write.format('csv').mode('overwrite').saveAsTable('{}.{}'.format('default', 'tablichka_numeric_csv'))

In [258]:
tt3.repartition(2,'rn_rec').write.format('csv').mode('overwrite').saveAsTable('{}.{}'.format('default', 'tablichka_numeric_csv_rep'))

In [255]:
tt3.printSchema()

root
 |-- egrul_org_id: string (nullable = true)
 |-- fld_dt: string (nullable = true)
 |-- rn_rec: long (nullable = true)



In [254]:
tt3 = tt3.withColumn('rn_rec', tt3.rn_rec.cast(LongType()))

In [81]:
# Тестовый дата-фрейм
cSchema = StructType([
    StructField('egrul_org_id', LongType(), True),
    StructField('rn_rec', LongType(), True),
    StructField('fld_dt', StringType(), True),
    ])

# Явное добавление записей в таблицу для удаления
data4del_lst = [
    Row(egrul_org_id='hjg', rn_rec='01', fld_dt='2009-09-25 00:00:00',), Row(egrul_org_id='uuu', rn_rec='02', fld_dt='2009-09-25 00:00:00',),
    Row(egrul_org_id='kjh', rn_rec='03', fld_dt='2009-09-25 00:00:00',), Row(egrul_org_id='aaa', rn_rec='04', fld_dt='2009-09-25 00:00:00',),
    Row(egrul_org_id='lij', rn_rec='01', fld_dt='2009-09-25 00:00:00',), Row(egrul_org_id='bbb', rn_rec='02', fld_dt='2009-09-25 00:00:00',),
    Row(egrul_org_id='fff', rn_rec='03', fld_dt='2013-10-11 00:00:00',), Row(egrul_org_id='ppp', rn_rec='04', fld_dt='2009-09-25 00:00:00',),
    Row(egrul_org_id='jjj', rn_rec='05', fld_dt='2009-09-25 00:00:00',),]
tt = sqlContext.createDataFrame(data4del_lst, schema=cSchema)
tt.orderBy('egrul_org_id', 'rn_rec').show()

+------------+------+-------------------+
|egrul_org_id|rn_rec|             fld_dt|
+------------+------+-------------------+
|           1|     1|2009-09-25 00:00:00|
|           1|     2|2009-09-25 00:00:00|
|           1|     3|2009-09-25 00:00:00|
|           1|     4|2009-09-25 00:00:00|
|           2|     1|2009-09-25 00:00:00|
|           2|     2|2009-09-25 00:00:00|
|           2|     3|2013-10-11 00:00:00|
|           2|     4|2009-09-25 00:00:00|
|           2|     5|2009-09-25 00:00:00|
+------------+------+-------------------+



In [82]:
tt.printSchema()

root
 |-- egrul_org_id: long (nullable = true)
 |-- rn_rec: long (nullable = true)
 |-- fld_dt: string (nullable = true)



In [83]:
tt.select("*", to_timestamp(tt.fld_dt, 'yyyy-MM-dd HH:mm:ss').alias('date_col')).write.mode('overwrite').saveAsTable('{}.{}'.format('default', 'tmp_test_sav'))

In [88]:
tt.select("*", to_timestamp(tt.fld_dt, 'yyyy-MM-dd HH:mm:ss').alias('date_col')).write.format('csv').mode('overwrite').saveAsTable('{}.{}'.format('default', '
                                                                                                                                                  '))

In [84]:
import pyspark.sql.functions as F

In [85]:
tt.select("*", to_timestamp(tt.fld_dt, 'yyyy-MM-dd HH:mm:ss').alias('timestamp'), F.to_date(F.col('fld_dt'), 'yyyy-MM-dd').alias('date'), F.month('fld_dt').alias('month')).show()
#.groupBy("rn_rec").count().where(F.col('count') == 2).where('rn_rec > 2').show()

+------------+------+-------------------+-------------------+----------+-----+
|egrul_org_id|rn_rec|             fld_dt|          timestamp|      date|month|
+------------+------+-------------------+-------------------+----------+-----+
|           1|     1|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           1|     2|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           1|     3|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           1|     4|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           2|     1|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           2|     2|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           2|     3|2013-10-11 00:00:00|2013-10-11 00:00:00|2013-10-11|   10|
|           2|     4|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
|           2|     5|2009-09-25 00:00:00|2009-09-25 00:00:00|2009-09-25|    9|
+------------+------+-------------------+-----------

In [69]:
tt.where(F.col('fld_dt') > '2010-01-01').show()

+------------+------+-------------------+
|egrul_org_id|rn_rec|             fld_dt|
+------------+------+-------------------+
|           2|     3|2013-10-11 00:00:00|
+------------+------+-------------------+



In [30]:
tt.createOrReplaceTempView('my_table')

In [11]:
# Сохраняем табл с бинарной разметкой ошибочных записей (чтобы не пересчитывать, возможно надо просто закешировать)
tbl_sav = '{}crt_table'.format(PREFIX_TMP)
print('{}.{}'.format(SCHEMA_SBX, tbl_sav))

sbx_t_team_graph_360.tmp_crt_table


In [12]:
sto=dt.now(); print('Start:\t', sto )
tt.write.format(SAVE_FORMAT).mode('overwrite').('{}.{}'.format(SCHEMA_SBX, tbl_sav))
et=dt.now(); dd=et-sto; print('Proc dt=', dd )

Start:	 2020-07-24 14:16:59.229146
Proc dt= 0:00:09.629188


In [215]:
df = sqlContext.read.format('csv').option('delimeter', ",").load('hdfs://nsld3/user/hive/warehouse/data_test/file1.csv', header=True)
df.show()

+---+-----------+-----------+
| ID|        INN|DESCRIPTION|
+---+-----------+-----------+
|  0|12134568175|     dsfdsy|
|  1|32134384832|     asdgfg|
|  2|41134324812|     asdsad|
+---+-----------+-----------+



In [194]:
#df_new = sqlContext.read.format('csv').option('delimeter', ",").option('header', True).load('hdfs://nsld3/user/hive/warehouse/data_test/file1.csv')

In [208]:
from pyspark.sql.types import LongType

In [224]:
modified_df = df.withColumn('INN', df.INN.cast(LongType()))

In [226]:
modified_df.show()

+---+-----------+-----------+
| ID|        INN|DESCRIPTION|
+---+-----------+-----------+
|  0|12134568175|     dsfdsy|
|  1|32134384832|     asdgfg|
|  2|41134324812|     asdsad|
+---+-----------+-----------+



In [227]:
df2 = sqlContext.read.format('csv').option('delimeter', ',').load('hdfs://nsld3/user/hive/warehouse/data_test/file2.csv', header=True)
df2.show()

+---+-----------+-----------+
| ID|        INN|DESCRIPTION|
+---+-----------+-----------+
|  0|02134568175|    1dsfdsy|
|  1|02134384832|    1asdgfg|
|  2|01134324812|    1asdsad|
+---+-----------+-----------+



In [236]:
modified_df2 = df2.withColumn('INN', df2.INN.cast(LongType()))

In [238]:
modified_df2.show()

+---+----------+-----------+
| ID|       INN|DESCRIPTION|
+---+----------+-----------+
|  0|2134568175|    1dsfdsy|
|  1|2134384832|    1asdgfg|
|  2|1134324812|    1asdsad|
+---+----------+-----------+



In [239]:
modified_df.write.format('csv').mode('overwrite').saveAsTable('{}.{}'.format('default', 'test_mod_csv_1'))
modified_df2.write.format('parquet').mode('overwrite').saveAsTable('{}.{}'.format('default', 'test_mod_parquet_20'))

In [24]:
# Останов
et = dt.now(); dd_ttl = et - stt
print('Общее время отработки\t {}  | dt={}'.format(et, dd_ttl))
sc.stop()

Общее время отработки	 2020-01-30 12:48:45.502338  | dt=0:16:09.172497


### Regex (Регулярные выражения)

#### regexp_extract()

In [357]:
sqlContext.sql(r"select regexp_extract('1123f', '(\\d+)')").show()

+-------------------------------+
|regexp_extract(1123f, (\d+), 1)|
+-------------------------------+
|                           1123|
+-------------------------------+



In [358]:
sqlContext.sql(r"select '11ss' as word,\
               regexp_extract('11ss', '(\\d+)') as digits, \
               regexp_extract('11ss', '(\\d+)(\\w+)',2) as strings ").show()

+----+------+-------+
|word|digits|strings|
+----+------+-------+
|11ss|    11|     ss|
+----+------+-------+



In [359]:
sqlContext.sql(r"select '11ss' as word,\
               regexp_extract('11ss', '(\\d+)') as digits, \
               regexp_replace('11ss', '(\\d+)(\\w+)', '$2') as strings ").show()

+----+------+-------+
|word|digits|strings|
+----+------+-------+
|11ss|    11|     ss|
+----+------+-------+



#### regexp_replace()

In [360]:
sqlContext.sql(r"select '11' as digit, regexp_replace('11', '\\d+', '**') as replaced").show()

+-----+--------+
|digit|replaced|
+-----+--------+
|   11|      **|
+-----+--------+



#### группы выражений вместе с regexp_extract()

In [449]:
sqlContext.sql(r"select '11ss' as word,\
               regexp_replace('11ss', '(\\d+)(\\w+)', '$1') as group_int, \
               regexp_replace('11ss', '(\\d+)(\\w+)', '$0') as group_all, \
               regexp_replace('11ss', '(\\d+)(\\w+)', '$2') as group_text").show()

+----+---------+---------+----------+
|word|group_int|group_all|group_text|
+----+---------+---------+----------+
|11ss|       11|     11ss|        ss|
+----+---------+---------+----------+



#### Rlike()

In [362]:
sqlContext.sql(r"SELECT CASE WHEN rlike('10', '^[24031]+$') THEN 'valid' ELSE 'invalid' END AS validator").show()

+---------+
|validator|
+---------+
|    valid|
+---------+



In [363]:
data_for_df = [(1, 'sedasdas  12321 f u320321 -3  kss'), (2, '4*^%^&(@*! (!DHSAJU)     _        (@#@dsfd))4')]

In [364]:
df_re = sc.parallelize(data_for_df).toDF(['id', 'text'])

In [365]:
df_re.printSchema()

root
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)



In [366]:
df_re.show(df_re.count(), False)

+---+---------------------------------------------+
|id |text                                         |
+---+---------------------------------------------+
|1  |sedasdas  12321 f u320321 -3  kss            |
|2  |4*^%^&(@*! (!DHSAJU)     _        (@#@dsfd))4|
+---+---------------------------------------------+



In [367]:
df_re.createOrReplaceTempView("df_re_table")

In [368]:
sqlContext.sql(r"select id, text as original_text, regexp_extract(text, '(\\d+)') as num_text from df_re_table" ).show(20, False)

+---+---------------------------------------------+--------+
|id |original_text                                |num_text|
+---+---------------------------------------------+--------+
|1  |sedasdas  12321 f u320321 -3  kss            |12321   |
|2  |4*^%^&(@*! (!DHSAJU)     _        (@#@dsfd))4|4       |
+---+---------------------------------------------+--------+



In [369]:
df_re_modified = sqlContext.sql(r"select id, \
               text as original_text, \
               regexp_extract(text, '(-?\\d+|[0-9]+)') as num_distinct, \
               regexp_extract(text, '(-\\d+)') as neg_num, \
               regexp_extract(text, '(\\D+)') as deleted_nums, \
               regexp_extract(text, '(\\w+)') as str_distinct,\
               regexp_replace(text, '(\\D+)', '') as num_full,\
               regexp_replace( regexp_replace(text, '(\\W+)', ''), '(_+)', '') as num_and_str,\
               regexp_replace( regexp_replace( regexp_replace(text, '(\\W+)', ''), '(\\d+)', ''), '(_)', '') as str_full,\
               regexp_replace(text, '( )', '') as no_space, \
               regexp_replace(text, '( +)', ' ') as no_duble_space \
               from df_re_table")


#русские буквы ЪьЁй

In [370]:
df_re_modified.show()

+---+--------------------+------------+-------+--------------------+------------+------------+--------------------+-------------+--------------------+--------------------+
| id|       original_text|num_distinct|neg_num|        deleted_nums|str_distinct|    num_full|         num_and_str|     str_full|            no_space|      no_duble_space|
+---+--------------------+------------+-------+--------------------+------------+------------+--------------------+-------------+--------------------+--------------------+
|  1|sedasdas  12321 f...|       12321|     -3|          sedasdas  |    sedasdas|123213203213|sedasdas12321fu32...|sedasdasfukss|sedasdas12321fu32...|sedasdas 12321 f ...|
|  2|4*^%^&(@*! (!DHSA...|           4|       |*^%^&(@*! (!DHSAJ...|           4|          44|        4DHSAJUdsfd4|   DHSAJUdsfd|4*^%^&(@*!(!DHSAJ...|4*^%^&(@*! (!DHSA...|
+---+--------------------+------------+-------+--------------------+------------+------------+--------------------+-------------+-----------

In [371]:
df_re_modified.show(df_re_modified.count(), False) 

+---+---------------------------------------------+------------+-------+-------------------------------------------+------------+------------+-------------------------+-------------+-------------------------------+----------------------------------+
|id |original_text                                |num_distinct|neg_num|deleted_nums                               |str_distinct|num_full    |num_and_str              |str_full     |no_space                       |no_duble_space                    |
+---+---------------------------------------------+------------+-------+-------------------------------------------+------------+------------+-------------------------+-------------+-------------------------------+----------------------------------+
|1  |sedasdas  12321 f u320321 -3  kss            |12321       |-3     |sedasdas                                   |sedasdas    |123213203213|sedasdas12321fu3203213kss|sedasdasfukss|sedasdas12321fu320321-3kss     |sedasdas 12321 f u320321 -3 kss   |


In [373]:
df_re_modified.printSchema()

root
 |-- id: long (nullable = true)
 |-- original_text: string (nullable = true)
 |-- num_distinct: string (nullable = true)
 |-- neg_num: string (nullable = true)
 |-- deleted_nums: string (nullable = true)
 |-- str_distinct: string (nullable = true)
 |-- num_full: string (nullable = true)
 |-- num_and_str: string (nullable = true)
 |-- str_full: string (nullable = true)
 |-- no_space: string (nullable = true)
 |-- no_duble_space: string (nullable = true)



In [372]:
df_re_modified.withColumn('len_simbols', F.length(df_re_modified.original_text))\
              .withColumn('len_simbols_no_space', F.length(df_re_modified.no_space))\
              .withColumn('len_simbols_no_double_space', F.length(df_re_modified.no_duble_space))\
              .select('original_text', 'len_simbols', 'len_simbols_no_space','len_simbols_no_double_space').show(20, False)

+---------------------------------------------+-----------+--------------------+---------------------------+
|original_text                                |len_simbols|len_simbols_no_space|len_simbols_no_double_space|
+---------------------------------------------+-----------+--------------------+---------------------------+
|sedasdas  12321 f u320321 -3  kss            |33         |26                  |31                         |
|4*^%^&(@*! (!DHSAJU)     _        (@#@dsfd))4|45         |31                  |34                         |
+---------------------------------------------+-----------+--------------------+---------------------------+



In [247]:
df_re_modified.createOrReplaceTempView('re_mod')

In [268]:
df_re_modified.rdd.map(lambda line: line.split()).map(len)

PythonRDD[941] at RDD at PythonRDD.scala:52

### Date
- timestamp
- date

In [400]:
date_data = [(1, '2020-09-15'), (2, '20-09-16 10:00:01')]

In [401]:
date_df = sc.parallelize(date_data).toDF(['id', 'date'])
date_df.show()

+---+-----------------+
| id|             date|
+---+-----------------+
|  1|       2020-09-15|
|  2|20-09-16 10:00:01|
+---+-----------------+



In [403]:
date_df.withColumn('date_mod', F.expr("to_date(date)")).show()

+---+-----------------+----------+
| id|             date|  date_mod|
+---+-----------------+----------+
|  1|       2020-09-15|2020-09-15|
|  2|20-09-16 10:00:01|      null|
+---+-----------------+----------+



In [477]:
date_df_mod = date_df.withColumn('date_mod', F.to_date('date'))\
                     .withColumn('timestamp_mod', F.to_timestamp('date', 'yy-MM-dd HH:mm:ss'))\
                     .withColumn('year', F.year('date_mod'))\
                     .withColumn('month', F.month('date_mod'))\
                     .withColumn('day_of_month', F.dayofmonth('date_mod'))\
                     .withColumn('hour', F.hour('timestamp_mod'))\
                     .withColumn('minute', F.minute('timestamp_mod'))\
                     .withColumn('second', F.second('timestamp_mod'))\
                     .withColumn('date_added', F.date_add('date_mod', 10))\
                     .withColumn('date_added_to_time_stamp', F.date_add('timestamp_mod', 10))\
                     .withColumn('difference', F.datediff('date_mod', 'date_added'))
# можно ли в условии указать несколько форматов? 
# формат - не желаемый вид, как в tsql, а исходный
# переделать через expr dateadd timestamp

In [478]:
date_df_mod.show()

+---+-----------------+----------+-------------------+----+-----+------------+----+------+------+----------+------------------------+----------+
| id|             date|  date_mod|      timestamp_mod|year|month|day_of_month|hour|minute|second|date_added|date_added_to_time_stamp|difference|
+---+-----------------+----------+-------------------+----+-----+------------+----+------+------+----------+------------------------+----------+
|  1|       2020-09-15|2020-09-15|               null|2020|    9|          15|null|  null|  null|2020-09-25|                    null|       -10|
|  2|20-09-16 10:00:01|      null|2020-09-16 10:00:01|null| null|        null|  10|     0|     1|      null|              2020-09-26|      null|
+---+-----------------+----------+-------------------+----+-----+------------+----+------+------+----------+------------------------+----------+



### более детальный DATEADD ( минуты, секунды, часы и тд) 
    F.expr('INTERVAL 30 minutes')

In [482]:
date_df_mod.withColumn('date_added_expr', date_df_mod.timestamp_mod + F.expr('INTERVAL 30 minutes')).show()
#  col

+---+-----------------+----------+-------------------+----+-----+------------+----+------+------+----------+------------------------+----------+-------------------+
| id|             date|  date_mod|      timestamp_mod|year|month|day_of_month|hour|minute|second|date_added|date_added_to_time_stamp|difference|    date_added_expr|
+---+-----------------+----------+-------------------+----+-----+------------+----+------+------+----------+------------------------+----------+-------------------+
|  1|       2020-09-15|2020-09-15|               null|2020|    9|          15|null|  null|  null|2020-09-25|                    null|       -10|               null|
|  2|20-09-16 10:00:01|      null|2020-09-16 10:00:01|null| null|        null|  10|     0|     1|      null|              2020-09-26|      null|2020-09-16 10:30:01|
+---+-----------------+----------+-------------------+----+-----+------------+----+------+------+----------+------------------------+----------+-------------------+



In [484]:
date_df_mod.withColumn('date_added_expr', date_df_mod.timestamp_mod + F.expr('INTERVAL 30 minutes')).printSchema()

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- date_mod: date (nullable = true)
 |-- timestamp_mod: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- second: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- date_added_to_time_stamp: date (nullable = true)
 |-- difference: integer (nullable = true)
 |-- date_added_expr: timestamp (nullable = true)



### Окна

In [10]:
wind_df = sc.parallelize([('001','2018-08-11',100),\
                          ('001','2015-07-04',100),\
                          ('001','2013-05-11',100),\
                          ('11111','2012-11-15',1001),\
                          ('11111','2007-08-01',2002),\
                          ('11111','2004-05-15',3003),\
                          ('11111','2002-03-16',4004),\
                          ('33','2020-03-12',2002),\
                          ('33','2019-01-31',3003),\
                          ('33','2019-01-31',4004),])\
                .toDF(['inn','from_dt','cur_val'])


'''--надо получить
inn;from_dt;to_dt;cur_val
'001';2018-08-11;2999-12-31;100
'001';2015-07-04;2018-08-10;100
'001';2013-05-11;2015-07-03;100
'11111';2012-11-15;2999-21-31;1001
'11111';2007-08-01;2012-11-14;2002
'11111';2004-05-15;2007-07-31;3003
'11111';2002-03-16;2004-05-14;4004
'33';2020-03-12;2999-12-312002
'33'2019-01-31;2020-03-11;3003
'33';2019-01-31;2019-01-31;4004'''


"--надо получить\ninn;from_dt;to_dt;cur_val\n'001';2018-08-11;2999-12-31;100\n'001';2015-07-04;2018-08-10;100\n'001';2013-05-11;2015-07-03;100\n'11111';2012-11-15;2999-21-31;1001\n'11111';2007-08-01;2012-11-14;2002\n'11111';2004-05-15;2007-07-31;3003\n'11111';2002-03-16;2004-05-14;4004\n'33';2020-03-12;2999-12-312002\n'33'2019-01-31;2020-03-11;3003\n'33';2019-01-31;2019-01-31;4004"

In [24]:
# добавил order by по cur_val, и доработал даты (-1)
wind_df.withColumn('to_dt', F.lead(wind_df['from_dt']).over(Window.partitionBy(wind_df.inn).orderBy('from_dt', 'cur_val')))\
       .withColumn('to_dt', F.when(col('from_dt') == col('to_dt'), col('to_dt')).otherwise(F.date_add(F.to_date('to_dt'), -1)))\
       .withColumn('to_dt', F.when(col('to_dt').isNull(), '2999-12-31').otherwise(col('to_dt')))\
       .select('inn', 'from_dt', 'to_dt', 'cur_val').orderBy('inn', 'to_dt').show()

+-----+----------+----------+-------+
|  inn|   from_dt|     to_dt|cur_val|
+-----+----------+----------+-------+
|  001|2013-05-11|2015-07-03|    100|
|  001|2015-07-04|2018-08-10|    100|
|  001|2018-08-11|2999-12-31|    100|
|11111|2002-03-16|2004-05-14|   4004|
|11111|2004-05-15|2007-07-31|   3003|
|11111|2007-08-01|2012-11-14|   2002|
|11111|2012-11-15|2999-12-31|   1001|
|   33|2019-01-31|2019-01-31|   3003|
|   33|2019-01-31|2020-03-11|   4004|
|   33|2020-03-12|2999-12-31|   2002|
+-----+----------+----------+-------+



### UDF
- Импортируем функционал udf в спарке
- Создаем саму функцию с нужным нам функцилоналом (в питоне)
- Регистрируем питон-функцию как UDF + возвращаемый тип значения, чтобы спарк мог ее использовать
- Применяем UDF в спарк-коде 

In [12]:
from pyspark.sql.functions import udf

In [13]:
def is_valid_inn(inn):
    if len(inn) < 4:
        return False
    return True
    

udf_is_valid_inn=udf(is_valid_inn, BooleanType())


In [14]:
wind_df.withColumn('is_correct_crc', udf_is_valid_inn(col('inn'))).show()

+-----+----------+-------+--------------+
|  inn|   from_dt|cur_val|is_correct_crc|
+-----+----------+-------+--------------+
|  001|2018-08-11|    100|         false|
|  001|2015-07-04|    100|         false|
|  001|2013-05-11|    100|         false|
|11111|2012-11-15|   1001|          true|
|11111|2007-08-01|   2002|          true|
|11111|2004-05-15|   3003|          true|
|11111|2002-03-16|   4004|          true|
|   33|2020-03-12|   2002|         false|
|   33|2019-01-31|   3003|         false|
|   33|2019-01-31|   4004|         false|
+-----+----------+-------+--------------+



### Broadcast filtering

In [60]:
filter_list = ['33', '001']

#### обычный фильтр

In [61]:
filter_df = wind_df.where(wind_df.inn.isin(filter_list))
filter_df.show()

+---+----------+-------+
|inn|   from_dt|cur_val|
+---+----------+-------+
|001|2018-08-11|    100|
|001|2015-07-04|    100|
|001|2013-05-11|    100|
| 33|2020-03-12|   2002|
| 33|2019-01-31|   3003|
| 33|2019-01-31|   4004|
+---+----------+-------+



In [62]:
filter_df.explain()

== Physical Plan ==
*Filter inn#707 IN (33,001)
+- Scan ExistingRDD[inn#707,from_dt#708,cur_val#709L]


#### broadcast фильтр

In [63]:
fbl = sc.broadcast(filter_list)

In [64]:
filter_droadcast_df = wind_df.where(wind_df.inn.isin(fbl.value))
filter_droadcast_df.show()

+---+----------+-------+
|inn|   from_dt|cur_val|
+---+----------+-------+
|001|2018-08-11|    100|
|001|2015-07-04|    100|
|001|2013-05-11|    100|
| 33|2020-03-12|   2002|
| 33|2019-01-31|   3003|
| 33|2019-01-31|   4004|
+---+----------+-------+



In [65]:
filter_droadcast_df.explain()

== Physical Plan ==
*Filter inn#707 IN (33,001)
+- Scan ExistingRDD[inn#707,from_dt#708,cur_val#709L]


In [25]:
# обязательно гасим контекст в конце , иначе он будет висеть и забирать ресурсы все время.
sc.stop()

#### HDFS

In [3]:
# вывод директорий или файлов в порядке возрастания даты. 
#Удобно, если хочется почистить старые таблицы.

!hdfs dfs -ls /data/adwh/prod_pim_dev_sbx/ | sort

In [ ]:
#Вывод файлов созданных до/после определенной даты.

!hdfs dfs -ls -t /data/adwh/prod_pim_dev_sbx/ | awk -v dateA="2021-11-01" '{if ($6 < dateA) {print $8}}'

In [5]:
# Удалить директорию или табличку из паркетов

!hdfs dfs -rmr -skipTrash /data/adwh/prod_pim_dev_sbx/*
!hdfs dfs -rmr -skipTrash /data/adwh/prod_pim_dev_sbx/